In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('survey_results_public.csv')
print(len(df))
df.head()

73268


,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Difficult,NaN
2,3,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Friend or family member...,Technical documentation;Blogs;Programming Game...,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,40205.0
3,4,I am a developer by profession,"Employed, full-time",Fully remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., Universit...",NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,215232.0
4,5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;O...,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN


In [5]:
# Filter the data to only keep profesionnal developers
print(len(df))
df_filtered = df[df['MainBranch'].str.contains("I am a developer by profession")]
print(len(df_filtered))

# Also filter to only relevants countries
def filter(df, column, filters):
    return df[df[column].isin(filters)]

countries = [
    'United States of America',
    'Switzerland',
    'United Kingdom of Great Britain and Northern Ireland',
    'France',
    'Germany',
    'Netherlands',
    'Spain',
    'Italy',
    'Austria',
    'Italy',
    'Canada',
    'Ireland',
    'Sweden',
    'Norway',
    'Australia',
    'Denmark'
]

df_filtered = filter(df_filtered,'Country',countries)

# df_filtered = df_filtered[df_filtered['Country'].str.contains("America")]

selectedColumns = [
    'MentalHealth',
    'DevType',
    'CompTotal',
    'CompFreq'
]

df_features = df_filtered[selectedColumns]
# Remove row that have NaN salary
df_features = df_features[df_features['CompTotal'].notna()]
df_features = df_features[df_features['CompFreq'].notna()]
df_features = df_features[df_features['MentalHealth'].notna()]

# Convert monthly salary to yearly, then drop the CompTotal and CompFreq columns
df_features['YearlySalary'] = df_features.apply(lambda row: int(row['CompTotal'] * 12) if row['CompFreq'] == "Monthly" else int(row['CompTotal']), axis=1)
df_features = df_features.drop(['CompTotal', 'CompFreq'] , axis=1)

# Filter outliers
min_yearly_salary = 0
max_yearly_salary = 1e7
df_features = df_features.loc[df_features['YearlySalary'] > min_yearly_salary]
df_features = df_features.loc[df_features['YearlySalary'] < max_yearly_salary]

print(len(df_features))

df_features

73268
53507
20261


,MentalHealth,DevType,YearlySalary
8,"Or, in your own words:","Developer, back-end",46000
10,None of the above,"Developer, full-stack;Developer, back-end",48000
12,None of the above,"Developer, full-stack",65000
14,None of the above,"Developer, full-stack;Academic researcher;DevO...",110000
17,None of the above,"Engineer, data",190000
...,...,...,...
73251,None of the above,"Developer, full-stack",65000
73253,None of the above,"Developer, back-end",121000
73261,None of the above,"Developer, full-stack",36000
73264,None of the above,Data scientist or machine learning specialist,107000


In [6]:
# df_features['MentalHealth'] = df_features['MentalHealth'].str.split(';')
# df_features = df_features.explode(['MentalHealth'])
# display(df_features['MentalHealth'].unique())
df_features['MentalHealth'].str.split(';').explode().unique()

# Only keep 'anxiety' and 'depression'
# mental_illnesses = ['I have an anxiety disorder', 
#                      'I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)']
# df_features = filter(df_features,'MentalHealth',mental_illnesses)

array(['Or, in your own words:', 'None of the above',
       'I have a mood or emotional disorder (e.g., depression, bipolar disorder, etc.)',
       'I have an anxiety disorder',
       'I have learning differences (e.g., Dyslexic, Dyslexia, etc.)',
       'Prefer not to say',
       'I have a concentration and/or memory disorder (e.g., ADHD, etc.)',
       "I have autism / an autism spectrum disorder (e.g. Asperger's, etc.)"],
      dtype=object)

In [8]:
df_features['MentalHealth'] = df_features['MentalHealth'].str.replace("I have a mood or emotional disorder \(e.g., depression, bipolar disorder, etc.\)", 'Depression', regex=True)
df_features['MentalHealth'] = df_features['MentalHealth'].str.replace("I have an anxiety disorder", 'Anxiety', regex=True)

df_features['MentalHealth'].str.split(';').explode().unique()

array(['Or, in your own words:', 'None of the above', 'Depression',
       'Anxiety',
       'I have learning differences (e.g., Dyslexic, Dyslexia, etc.)',
       'Prefer not to say',
       'I have a concentration and/or memory disorder (e.g., ADHD, etc.)',
       "I have autism / an autism spectrum disorder (e.g. Asperger's, etc.)"],
      dtype=object)

In [9]:
df_features = df_features.dropna()

In [10]:
df_features

,MentalHealth,DevType,YearlySalary
8,"Or, in your own words:","Developer, back-end",46000
10,None of the above,"Developer, full-stack;Developer, back-end",48000
12,None of the above,"Developer, full-stack",65000
14,None of the above,"Developer, full-stack;Academic researcher;DevO...",110000
17,None of the above,"Engineer, data",190000
...,...,...,...
73251,None of the above,"Developer, full-stack",65000
73253,None of the above,"Developer, back-end",121000
73261,None of the above,"Developer, full-stack",36000
73264,None of the above,Data scientist or machine learning specialist,107000


In [11]:
# Export the CSV
df_features.to_csv('mental_health.csv', sep=',', encoding='utf-8')

## Test the visualization

In [14]:
# Get the proportions mental illness by DevType
# df_mh = pd.DataFrame()
# df_mh.set_index(df_features['DevType'].unique())
# df_mh['Anxiety'] = df_features['MentalHealth'].str.count('Anxiety', normalize=True) * 100
# dev_type_list = df_features['DevType'].str.split(';').dropna().explode().unique().tolist()

In [50]:
DevType = [
    'Developer, full-stack',
    'Academic researcher',
    'DevOps specialist',
    'Developer, back-end',
    'Developer, desktop or enterprise applications',
    'Developer, embedded applications or devices',
    'Developer, game or graphics',
    'Developer, front-end',
    'Engineer, data',
    'Developer, QA or test',
    'Developer, mobile',
    'Database administrator',
    'Cloud infrastructure engineer',
    'Data or business analyst',
    'Designer',
    'Marketing or sales professional',
    'Data scientist or machine learning specialist',
    'Security professional',
    'Project manager',
    'Senior Executive (C-Suite, VP, etc.)',
    'Engineering manager',
    'System administrator',
    'Scientist',
    'Product manager',
    'Engineer, site reliability',
    'Educator',
    'Other',
    'Blockchain',
    'Student'
]

def filter_multicat(df : pd.DataFrame, column, filter) -> pd.DataFrame:
    return df[df[column].str.contains(filter)]

def aggregate_health(data, mental_health_type: str):
    output = pd.DataFrame()
    numbers = []
    for devType in DevType:
        df = filter_multicat(data, 'DevType', devType)
        total = df.shape[0]
        # print(total)
        # print(filter_multicat(df, 'MentalHealth', mental_health_type).size)
        if total > 0:
            numbers.append(filter_multicat(df, 'MentalHealth', mental_health_type).shape[0]/total)
        else:
            numbers.append(0)
    output['DevType'] = DevType
    output[mental_health_type] = numbers
    return output

def getHealthOutput(data, salary_min, salary_max, mental_health_type):
    df = data
    # Filter the salary
    df = df.loc[df['YearlySalary'] > salary_min]
    df = df.loc[df['YearlySalary'] < salary_max]

    # Get the proportion of the data for each DevType
    return aggregate_health(df, mental_health_type)

df = getHealthOutput(df_features, 80e3,160e3, 'Depression')
df = df.sort_values(by=['Depression'])
df = df.reset_index(drop=True)
df

C:\Users\Antony\AppData\Local\Temp\ipykernel_9244\4294898406.py:34: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



,DevType,Depression
0,"Senior Executive (C-Suite, VP, etc.)",0.000000
1,Student,0.076923
2,Marketing or sales professional,0.086957
3,Data scientist or machine learning specialist,0.099359
4,Project manager,0.101744
5,"Developer, embedded applications or devices",0.109409
6,"Developer, mobile",0.112164
7,Cloud infrastructure engineer,0.112752
8,"Engineer, data",0.115217
9,System administrator,0.115304


In [84]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# df = px.data.stocks()
fig = make_subplots(rows=1, cols=2)
fig.append_trace(go.Bar(x=df["Depression"], y=df["DevType"], orientation='h'), row=1, col=1)
fig.append_trace(go.Bar(x=df["Depression"], y=df["DevType"], orientation='h'), row=1, col=2)
fig.update_xaxes(autorange='reversed', col=1, showticklabels=False)
fig.update_yaxes(col=2, showticklabels=False)
fig.update_layout(polar = dict(radialaxis = dict(showticklabels = False)))
# fig.update_layout(bargap=0.2)
# fig.update_traces(width=1)
fig.show()